In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Customer Churn Analysis and Data Augmentation Notebook

## README

This notebook performs customer churn analysis and data augmentation on a customer dataset. It includes the following main components:

1. Data Loading: Loads the original training and testing datasets.
2. Feature Engineering: Adds new features to the dataset to enhance the analysis.
3. Data Augmentation: Creates synthetic data to expand the dataset, including:
   - Adding noise to numerical features
   - Introducing anomalies
   - Adding missing values
4. Visualization: Generates various plots to compare the original and augmented datasets, including:
   - Distribution comparisons
   - Correlation heatmaps
   - Churn rate by categorical features
   - Feature importance
   - Scatter plots and box plots of key features

### How to Use

1. Ensure you have the required libraries installed (see requirements.txt).
2. Update the file paths in the `main()` function to point to your data files.
3. Run the entire notebook.
4. Examine the printed data samples and shapes, as well as the generated visualizations.

### Key Functions

- `load_data()`: Loads the datasets
- `add_features()`: Adds new features to the data
- `augment_data()`: Creates an augmented dataset
- `create_visualizations()`: Generates various plots for analysis

# Read in data and create additional features

In [4]:
# Load the datasets
train_data_path = '../data/customer_churn_dataset-training-master.csv'
test_data_path = '../data/customer_churn_dataset-testing-master.csv'


train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)


In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [7]:
def add_features(data):
    """
    Add new features to the dataset.
    """
    # Tenure in Years
    data['Tenure_Years'] = data['Tenure'] / 12
    
    # Monthly Spend
    data['Monthly_Spend'] = data['Total Spend'] / data['Tenure']
    
    # Interaction Frequency
    data['Interaction_Frequency'] = data['Usage Frequency'] / data['Tenure']
    
    # Support Call Rate
    data['Support_Call_Rate'] = data['Support Calls'] / data['Tenure']
    
    # Payment Delay Rate
    data['Payment_Delay_Rate'] = data['Payment Delay'] / data['Tenure']
    
    # Customer Loyalty
    data['Loyal_Customer'] = data['Tenure'].apply(lambda x: 1 if x > 24 else 0)
    
    return data

# Sample data and create synthetic data with noise and additional missing values - to make it a bit more intersting

In [8]:
# Set a random seed for reproducibility
np.random.seed(42)

In [ ]:
def create_synthetic_data(original_data, synth_size):
    """
    Create synthetic data by sampling from the original data and adding noise.
    """
    # Sample rows from the original data with replacement
    synthetic_data = original_data.sample(n=synth_size, replace=True).reset_index(drop=True)
    
    # Add noise to numerical columns
    noise_factor = 0.05  # 5% random noise
    numeric_columns = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls', 'Payment Delay', 
                       'Total Spend', 'Last Interaction', 'Tenure_Years', 'Monthly_Spend', 
                       'Interaction_Frequency', 'Support_Call_Rate', 'Payment_Delay_Rate']
    
    for column in numeric_columns:
        synthetic_data[column] = synthetic_data[column] * (1 + noise_factor * np.random.randn(synth_size))
    
    return synthetic_data

In [ ]:
def introduce_anomalies(data, anomaly_ratio=0.01):
    """
    Introduce anomalies by setting extreme values for certain features.
    """
    anomaly_indices = np.random.choice(len(data), size=int(anomaly_ratio * len(data)), replace=False)
    data.loc[anomaly_indices, 'Support Calls'] = data['Support Calls'].max() * 5
    data.loc[anomaly_indices, 'Total Spend'] = data['Total Spend'].max() * 10
    return data

In [ ]:
def introduce_missing_values(data, missing_ratio=0.03):
    """
    Introduce missing values in specified columns.
    """
    columns_with_missing = ['Age', 'Gender', 'Tenure', 'Payment Delay', 'Last Interaction']
    for column in columns_with_missing:
        missing_indices = np.random.choice(len(data), size=int(missing_ratio * len(data)), replace=False)
        data.loc[missing_indices, column] = np.nan
    return data

In [ ]:
def augment_data(original_data, augmentation_factor=9):
    """
    Create an augmented dataset by combining original and synthetic data.
    """
    synth_size = len(original_data) * augmentation_factor
    synthetic_data = create_synthetic_data(original_data, synth_size)
    synthetic_data = introduce_anomalies(synthetic_data)
    synthetic_data = introduce_missing_values(synthetic_data)
    augmented_data = pd.concat([original_data, synthetic_data], axis=0).reset_index(drop=True)
    return augmented_data

In [9]:
# Add new features
train_data = add_features(train_data)
test_data = add_features(test_data)

# Create augmented dataset
augmented_data = augment_data(train_data)

# Print sample of original and augmented data
print("Original Data Sample:")
print(train_data.head())
print("\nAugmented Data Sample:")
print(augmented_data.head())

# Print shape of original and augmented data
print(f"\nOriginal Data Shape: {train_data.shape}")
print(f"Augmented Data Shape: {augmented_data.shape}")

augmented_data.to_csv("../data/augmented-training data.csv")

In [ ]:
augmented_data.head()

In [26]:
augmented_data.to_csv("../data/augmented-training data.csv")

# Compare the original and augmented dataset

In [ ]:
# Set the style for all plots
plt.style.use('seaborn')

In [16]:
def compare_distributions(original_data, augmented_data, columns, n_rows, n_cols):
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
    axes = axes.flatten()
    
    for i, col in enumerate(columns):
        sns.kdeplot(data=original_data, x=col, ax=axes[i], label='Original', color='blue')
        sns.kdeplot(data=augmented_data, x=col, ax=axes[i], label='Augmented', color='red')
        axes[i].set_title(f'Distribution of {col}')
        axes[i].legend()
    
    plt.tight_layout()
    plt.show()

In [17]:
def plot_correlation_heatmap(data, title):
    plt.figure(figsize=(12, 10))
    corr = data.select_dtypes(include=[np.number]).corr()
    sns.heatmap(corr, annot=False, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title(title)
    plt.show()

In [18]:
def plot_churn_rate_by_category(data, category, title):
    plt.figure(figsize=(10, 6))
    sns.barplot(x=category, y='Churn', data=data, estimator=lambda x: (x == 1).mean() * 100)
    plt.title(title)
    plt.ylabel('Churn Rate (%)')
    plt.xticks(rotation=45)
    plt.show()


In [19]:
def plot_feature_importance(data):

    # Prepare the data
    X = data.drop('Churn', axis=1)
    y = data['Churn']

    # Encode categorical variables
    le = LabelEncoder()
    X['Gender'] = le.fit_transform(X['Gender'])

    # Train a Random Forest model
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X, y)

    # Plot feature importance
    feature_importance = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_})
    feature_importance = feature_importance.sort_values('importance', ascending=False)

    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance)
    plt.title('Feature Importance')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.show()

In [ ]:
# Compare distributions of key numerical features
numerical_features = ['Age', 'Tenure', 'Usage Frequency', 'Total Spend', 'Monthly_Spend', 'Interaction_Frequency']
compare_distributions(train_data, augmented_data, numerical_features, 2, 3)

# Plot correlation heatmaps
plot_correlation_heatmap(train_data, 'Correlation Heatmap - Original Data')
plot_correlation_heatmap(augmented_data, 'Correlation Heatmap - Augmented Data')

In [ ]:
# Plot churn rate by categorical features
categorical_features = ['Gender', 'Loyal_Customer']
for feature in categorical_features:
    plot_churn_rate_by_category(train_data, feature, f'Churn Rate by {feature} - Original Data')
    plot_churn_rate_by_category(augmented_data, feature, f'Churn Rate by {feature} - Augmented Data')


In [ ]:
# Scatter plot of Total Spend vs Tenure, colored by Churn
plt.figure(figsize=(12, 8))
sns.scatterplot(data=train_data, x='Total Spend', y='Tenure', hue='Churn', alpha=0.6)
plt.title('Total Spend vs Tenure (Original Data)')
plt.show()

plt.figure(figsize=(12, 8))
sns.scatterplot(data=augmented_data, x='Total Spend', y='Tenure', hue='Churn', alpha=0.6)
plt.title('Total Spend vs Tenure (Augmented Data)')
plt.show()


In [ ]:
# Box plot of Monthly Spend by Churn status
plt.figure(figsize=(10, 6))
sns.boxplot(x='Churn', y='Monthly_Spend', data=train_data)
plt.title('Monthly Spend by Churn Status (Original Data)')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Churn', y='Monthly_Spend', data=augmented_data)
plt.title('Monthly Spend by Churn Status (Augmented Data)')
plt.show()